In [1]:
from __future__ import division, print_function, absolute_import

# Import tflearn and some helpers
import tflearn 
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import pickle

dataset_path="C:\\Users\\smith\\Documents\\Becky\\HawkWatch\\Images\\"
output_path="C:\\Users\\smith\\Documents\\Becky\\HawkWatch\\bird-sandbox\\"


C:\Users\smith\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# Load the data set
# BAC-added argument encoding='bytes'
X, Y, X_test, Y_test = pickle.load(open(dataset_path+"full_dataset.pkl", "rb"), encoding='bytes')

# Shuffle the data
X, Y = shuffle(X, Y)


In [3]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()


In [4]:
# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)
print("***** Flip, rotate and blur the images")

***** Flip, rotate and blur the images


In [5]:
# Clear extra nodes caused by restarts
# figure out best place to put this, maybe before the DNN call

#from tensorflow.python.framework import ops
#ops.reset_default_graph()
# The call to DNN gave this error:
# ValueError: Cannot use the given session to evaluate tensor: the tensor's graph is different from the session's graph.
# probably due to resetting the default graph
#print("***** Default graph reset")

In [6]:
# Define our network architecture:

# Input is a 32x32 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
print("***** network created: ", network)

***** network created:  Tensor("InputData/X:0", shape=(?, 32, 32, 3), dtype=float32)


In [7]:
# Step 1: Convolution
network = conv_2d(network, 32, 3, activation='relu')

# Step 2: Max pooling
network = max_pool_2d(network, 2)
print("***** Steps 1 and 2")

***** Steps 1 and 2


In [8]:
# Step 3: Convolution again
network = conv_2d(network, 64, 3, activation='relu')

# Step 4: Convolution yet again
network = conv_2d(network, 64, 3, activation='relu')

print("***** Steps 3 and 4")

***** Steps 3 and 4


In [9]:
# Step 5: Max pooling again
network = max_pool_2d(network, 2)

# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')

print("***** Steps 5 and 6")

***** Steps 5 and 6


In [10]:
# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

# Step 8: Fully-connected neural network with two outputs (0=isn't a bird, 1=is a bird) to make the final prediction
network = fully_connected(network, 2, activation='softmax')
print("***** Steps 7 and 8")

***** Steps 7 and 8


In [11]:
# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)


print("***** regression")

***** regression


In [12]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=3, checkpoint_path='bird-classifier.tfl.ckpt')

print("***** Created DNN model")

***** Created DNN model


In [13]:
# Train it! We'll do 100 training passes and monitor it as it goes.
#
# BAC-Let's start with fewertraining passes until everything works
#
#num_training_passes=100  # Adam's value
num_training_passes=70

model.fit(X, Y, n_epoch=num_training_passes, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
          run_id='bird-classifier')

Training Step: 41439  | total loss: 0.16758 | time: 372.993s
| Adam | epoch: 070 | loss: 0.16758 - acc: 0.9360 -- iter: 56736/56780
Training Step: 41440  | total loss: 0.17827 | time: 396.027s
| Adam | epoch: 070 | loss: 0.17827 - acc: 0.9319 | val_loss: 0.19868 - val_acc: 0.9416 -- iter: 56780/56780
--
INFO:tensorflow:C:\Users\smith\Documents\Becky\HawkWatch\bird-sandbox\bird-classifier.tfl.ckpt-41440 is not in all_model_checkpoint_paths. Manually adding it.


In [14]:
# Save model when training is complete to a file

model.save("bird-classifier.tfl")
print("Network trained and saved as bird-classifier.tfl!")


INFO:tensorflow:C:\Users\smith\Documents\Becky\HawkWatch\bird-sandbox\bird-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.
Network trained and saved as bird-classifier.tfl!


In [15]:
# Save model when training is complete to a file
#model.save("another-bird-classifier.tfl")
#print("Network trained and saved as another-bird-classifier.tfl!")